### This file has code for Pricing Options with the LSMC (Least-Square-Monte-Carlo) Method
    1. Using Laguerre polynomials
    2. Hermite polynomials
    3. Simple Monomials

In [ ]:
import numpy as np
from numpy import sqrt, log, sin, cos, exp, pi, mean, repeat, var, matmul, transpose
from scipy.stats import norm
import matplotlib.pyplot as plt
from math import factorial as f
from datetime import date
import pandas_datareader as pdr
import pandas as pd
import os
from itertools import permutations, combinations_with_replacement, product

import progressbar
import time

In [ ]:
"""
@param: Lfunction: Monomials =1 Hermite = 2, Laguerre = 3
@param: k = number of terms in Lfunction
@param So = initial stock price
@param strike = strike price
@output: Option price

"""
def LSMC(So, strike, sigma, r, T, delta, paths, Lfunction, k):
    # Simulating stocks path
    
    N = int(T/delta)
    z1 = np.random.randn(int(paths/2+1), N)
    z2 = -z1
    Z = np.concatenate((z1, z2), axis=0)
    S = np.zeros((paths, N+1))
    S[:, 0] = So/strike
    for i in range(paths):
      for j in range(N):
          S[i, j+1] = S[i, j] + r*S[i, j]*delta + sigma*S[i, j]*sqrt(delta)*Z[i, j]
    
    #Index matrix, calculating value at the last column
    index = np.zeros((paths, N))
    index[:, N-1] = np.where( (1-S[:, N]) > 0, 1, 0 )
    
    
    #Backward loop to set index 1/0 based on optimum exercise time
    for n in range(N-1, 0, -1):
        
        X = S[:, n]
        #Taking in the money path only
        payoffNow = np.where(1-X>0, 1-X, 0)
        ITM_paths = payoffNow>0
        X = X[ITM_paths]
        Y = np.zeros((paths))
        j = 1
        for i in range(n, N, 1):
            Y = Y+exp(-r*j*delta)*np.multiply(index[:, i], 1-S[:, i+1])
            j=j+1 
        #Taking Y for in the money path only
        Y = Y[ITM_paths]
                
        if(Lfunction == 1): #Monomials
            fmat = np.stack( ( (k>=1)*np.repeat(1, X.shape[0]), (k>=2)*X, (k>=3)*X**2, (k>=4)*X**3 ), axis=0 )
            fmat = fmat[~np.all(fmat==0, axis=1)]

        if(Lfunction == 2): #Hermite
            fmat = np.stack( ((k>=1)*np.repeat(1, X.shape[0]), (k>=2)*2*X, (k>=3)*(4*X**2-2), (k>=4)*(8*X**3 - 12*X)), axis=0 )
            fmat = fmat[~np.all(fmat==0, axis=1)]

        if(Lfunction == 3): #Laguerre
            fmat = np.stack( ((k>=1)*exp(-X/2), (k>=2)*exp(-X/2)*(1-X), (k>=3)*exp(-X/2)*(1-2*X+X**2/2), (k>=4)*exp(-X/2)*(1-3*X+3*X**2/2-X**3/6)), axis=0 )
            fmat = fmat[~np.all(fmat==0, axis=1)]

        b = matmul(fmat, Y)
       
        A = matmul(fmat, transpose(fmat)) 
        a = np.linalg.solve(A, b)
        ECV = np.zeros(paths)
        ECV[ITM_paths] = matmul(transpose(a), fmat)
        index[:, n-1] = np.where( (1-S[:, n]) >= ECV, 1, 0 ) 
        index[:, n:N] = index[:, n:N]*(index[:, n-1]==0).reshape(paths, 1)
        
    #constructing discounting array
    discountArray = np.arange(N)
    discountArray = np.tile(discountArray, (paths,1))
    discountArray = exp(-r*delta*(discountArray+1))
    Price = strike*(1/paths)*np.sum(np.multiply(1-S[:, 1:N+1], discountArray*index))
    return Price
 
############### Ques 1.a ###################
#inputs from question
sigma = 0.20
r = 0.06
paths = 10000
N=100 #time division
strike = 40


TSok = [[0.5, 1, 2], [36, 40, 44], [2, 3, 4]]
inputs = np.array(list(product(*TSok)))
price = np.zeros(27)
Lfunction=3 #Laguerre
#to check progress
bar = progressbar.ProgressBar(redirect_stdout=True)

for i in range(inputs.shape[0]):
    T = inputs[i, 0]
    delta = T/N
    So = inputs[i, 1]
    k = inputs[i, 2]
    price[i] = LSMC(So, strike, sigma, r, T, delta, paths, Lfunction, k)
    bar.update(i*100/26)

price_1a = np.insert(inputs, 3, price, axis=1)
priceDataFrame = pd.DataFrame(price_1a, columns=['T', 'So', 'k', "Laguerre"])
            
############### Ques 1.b ###################
#to check progress
bar = progressbar.ProgressBar(redirect_stdout=True)
price = np.zeros(27)
Lfunction=2 #Hermite
for i in range(inputs.shape[0]):
    
    T = inputs[i, 0]
    delta = T/N
    So = inputs[i, 1]
    k = inputs[i, 2]
    price[i] = LSMC(So, strike, sigma, r, T, delta, paths, Lfunction, k)
    bar.update(i*100/26)
    
priceDataFrame.insert(4, column='Hermite', value = price)  
  
        
############### Ques 1.c ###################
#to check progress
bar = progressbar.ProgressBar(redirect_stdout=True)
price = np.zeros(27)
Lfunction=1 #Monomials
for i in range(inputs.shape[0]):
    T = inputs[i, 0]
    delta = T/N
    So = inputs[i, 1]
    k = inputs[i, 2]
    price[i] = LSMC(So, strike, sigma, r, T, delta, paths, Lfunction, k)
    bar.update(i*100/26)

priceDataFrame.insert(5, column='Monomials', value = price) 



In [ ]:

##################### Ques 2 a ###########################
#simulate stock paths
So = 65
sigma = .20
r = 0.06
t = 0.2
T=1

paths = 100000
N = 100
delta = T/N
z1 = np.random.randn(int(paths/2+1), N)
z2 = -z1
Z = np.concatenate((z1, z2), axis=0)
S = np.zeros((paths, N+1))
S[:, 0] = So
for i in range(paths):
  for j in range(N):
      S[i, j+1] = S[i, j] + r*S[i, j]*delta + sigma*S[i, j]*sqrt(delta)*Z[i, j] 

ST = S[:, N]
St = S[:, int(0.2*N)]
payoff = np.where(St-ST>0, St-ST, 0)

EurPutPrice = exp(-r*T)*mean(payoff)


##################### Ques 2 b ###########################


"""
@param extra parameter t is the forward time when
American Put option strike price would be calculated
@param: Lfunction: Monomials =1 Hermite = 2, Laguerre = 3
@param: k = number of terms in Lfunction
@param So = initial stock price
@param strike = strike price
@output: Option price
"""
def LSMC_FwdStart(So, t, sigma, r, T, delta, paths, Lfunction, k):
    # Simulating stocks path
    
    N = int(T/delta)
    z1 = np.random.randn(int(paths/2+1), N)
    z2 = -z1
    Z = np.concatenate((z1, z2), axis=0)
    S = np.zeros((paths, N+1))
    S[:, 0] = So
    for i in range(paths):
      for j in range(N):
          S[i, j+1] = S[i, j] + r*S[i, j]*delta + sigma*S[i, j]*sqrt(delta)*Z[i, j]
    
    strike = S[:, int(t*N)].reshape(paths, 1)
    S = S/strike
    
    #Index matrix, calculating value at the last column
    index = np.zeros((paths, N))
    index[:, N-1] = np.where( (1-S[:, N]) > 0, 1, 0 )
    
    #Backward loop to set index 1/0 based on optimum exercise time
    for n in range(N-1, int(t*N), -1):
        X = S[:, n]
        #Taking in the money path only
        payoffNow = np.where(1-X>0, 1-X, 0)
        ITM_paths = payoffNow>0
        X = X[ITM_paths]
        Y = np.zeros((paths))
        j = 1
        for i in range(n, N, 1):
            Y = Y+exp(-r*j*delta)*np.multiply(index[:, i], 1-S[:, i+1])
            j=j+1
            
        #Taking Y for in the money path only
        Y = Y[ITM_paths]
                
        if(Lfunction == 1): #Monomials
            fmat = np.stack( ( (k>=1)*np.repeat(1, X.shape[0]), (k>=2)*X, (k>=3)*X**2, (k>=4)*X**3 ), axis=0 )
            fmat = fmat[~np.all(fmat==0, axis=1)]

        if(Lfunction == 2): #Hermite
            fmat = np.stack( ((k>=1)*np.repeat(1, X.shape[0]), (k>=2)*2*X, (k>=3)*(4*X**2-2), (k>=4)*(8*X**3 - 12*X)), axis=0 )
            fmat = fmat[~np.all(fmat==0, axis=1)]

        if(Lfunction == 3): #Laguerre
            fmat = np.stack( ((k>=1)*exp(-X/2), (k>=2)*exp(-X/2)*(1-X), (k>=3)*exp(-X/2)*(1-2*X+X**2/2), (k>=4)*exp(-X/2)*(1-3*X+3*X**2/2-X**3/6)), axis=0 )
            fmat = fmat[~np.all(fmat==0, axis=1)]

        b = matmul(fmat, Y)
       
        A = matmul(fmat, transpose(fmat)) 
        a = np.linalg.solve(A, b)
        ECV = np.zeros(paths)
        ECV[ITM_paths] = matmul(transpose(a), fmat)
        index[:, n-1] = np.where( (1-S[:, n]) >= ECV, 1, 0 ) 
        index[:, n:N] = index[:, n:N]*(index[:, n-1]==0).reshape(paths, 1)
       
    #constructing discounting array
    discountArray = np.arange(N)
    discountArray = np.tile(discountArray, (paths,1))
    discountArray = exp(-r*delta*(discountArray+1))
    scaledPriceEachPath=np.sum(np.multiply(1-S[:, 1:N+1], discountArray*index), axis=1).reshape(paths, 1)
    Price = (1/paths)*np.sum(np.multiply(scaledPriceEachPath, strike))
    return Price

So = 65
t=0.2
sigma=0.2
r = 0.06
T = 1
N = 100
delta = T/N
paths = 100000
Lfunction=1
k=3

AmericanFwdPut = LSMC_FwdStart(So, t, sigma, r, T, delta, paths, Lfunction, k)     
